## Allstate week 5

There are a couple of new foundings that I'd like to share with you:

1. Two-way interactions of categorical features
2. Official LightGBM Python wrapper - 4x faster than XGBoost
3. Additional tuning tips

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn import preprocessing, pipeline, metrics, grid_search, cross_validation
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_absolute_error

from scipy import sparse
%matplotlib inline

In [ ]:
def log_mae(labels,preds,lift=200):
    return mean_absolute_error(np.exp(labels)-lift, np.exp(preds)-lift)

def logregobj(labels, preds):
    con = 2
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess 


log_mae_scorer = metrics.make_scorer(log_mae, greater_is_better = False)

def search_model(train_x, train_y, est, param_grid, n_jobs, cv, refit=False):
##Grid Search for the best model
    model = grid_search.GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring    = log_mae_scorer,
                                     verbose    = 10,
                                     n_jobs  = n_jobs,
                                     iid        = True,
                                     refit    = refit,
                                     cv      = cv)
    # Fit Grid Search Model
    model.fit(train_x, train_y)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:", model.best_params_)
    print("Scores:", model.grid_scores_)
    return model

def xg_eval_mae(yhat, dtrain, lift=200):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y)-lift, np.exp(yhat)-lift)

def xgb_logregobj(preds, dtrain):
    con = 2
    labels = dtrain.get_label()
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess


def search_model_mae (train_x, train_y, est, param_grid, n_jobs, cv, refit=False):
##Grid Search for the best model
    model = grid_search.GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring    = 'neg_mean_absolute_error',
                                     verbose    = 10,
                                     n_jobs  = n_jobs,
                                     iid        = True,
                                     refit    = refit,
                                     cv      = cv)
    # Fit Grid Search Model
    model.fit(train_x, train_y)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:", model.best_params_)
    print("Scores:", model.grid_scores_)
    return model

In [ ]:
from sklearn.cross_validation import StratifiedKFold, KFold
def xgb_blend(estimators, train_x, train_y, test_x, fold, early_stopping_rounds=0):
    print ("Blend %d estimators for %d folds" % (len(estimators), fold))
    skf = list(KFold(len(train_y), fold))
    
    train_blend_x = np.zeros((train_x.shape[0], len(estimators)))
    test_blend_x = np.zeros((test_x.shape[0], len(estimators)))
    scores = np.zeros ((len(skf),len(estimators)))
    best_rounds = np.zeros ((len(skf),len(estimators)))
    

    
    for j, est in enumerate(estimators):
        print ("Model %d: %s" %(j+1, est))
        test_blend_x_j = np.zeros((test_x.shape[0], len(skf)))
        for i, (train, val) in enumerate(skf):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x[train]
            train_y_fold = train_y[train]
            val_x_fold = train_x[val]
            val_y_fold = train_y[val]
            if early_stopping_rounds==0: # without early stopping
                est.fit(train_x_fold, train_y_fold)
                best_rounds[i,j]=est.n_estimators
                val_y_predict_fold = est.predict(val_x_fold)
                score = log_mae(val_y_fold, val_y_predict_fold,200)
                print ("Score: ", score)
                scores[i,j]=score
                train_blend_x[val, j] = val_y_predict_fold
                test_blend_x_j[:,i] = est.predict(test_x)
                print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))
            else:                        # early stopping
                est.set_params( n_estimators=10000)
                est.fit(train_x_fold,
                        train_y_fold,
                        eval_set=[(val_x_fold, val_y_fold)],
                        eval_metric=xg_eval_mae,
                        early_stopping_rounds=early_stopping_rounds,
                        verbose=False
                       )
                best_round=est.best_iteration
                best_rounds[i,j]=best_round
                print ("best round %d" % (best_round))
                val_y_predict_fold = est.predict(val_x_fold,ntree_limit=best_round)
                score = log_mae(val_y_fold, val_y_predict_fold,200)
                print ("Score: ", score)
                scores[i,j]=score
                train_blend_x[val, j] = val_y_predict_fold
                test_blend_x_j[:,i] = est.predict(test_x,ntree_limit=best_round)
                print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            
   
        test_blend_x[:,j] = test_blend_x_j.mean(1)
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x, scores,best_rounds )

## LightGBM official Python Wrapper

last week Microsoft released the official Python Wrapper for LightGBM. Following are the key benifit:

* Scikit-learn style API
* No need to dump training data into disk (as opposed to the pyLightGBM wrapper we used before). As a result, you can expect a huge improvement on speed - at least 4X faster than XGBoost
* Support sparse matrix
* Support custom objective function and metric. You can used log-trainsformed MAE for training now!
* Native cross validation function makes validation much easier


https://github.com/Microsoft/LightGBM/tree/master/python-package

#### Installation guide:

Following Installation Guide to build first. 

For windows users, please change the build config to DLL.

Install with cd python-package; python setup.py install
Note: Make sure you have setuptools


### Below are the updated utitily functions based on official wrapper

In [ ]:
import lightgbm as lgb
def lgbm_eval_mae(yhat, dtrain, lift=200):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y)-lift, np.exp(yhat)-lift), False

from sklearn.cross_validation import StratifiedKFold, KFold
def lgbm_blend(estimators, train_x, train_y, test_x, fold, early_stopping_rounds=0):
    print ("Blend %d estimators for %d folds" % (len(estimators), fold))
    skf = list(KFold(len(train_y), fold))
    
    train_blend_x = np.zeros((train_x.shape[0], len(estimators)))
    test_blend_x = np.zeros((test_x.shape[0], len(estimators)))
    scores = np.zeros ((len(skf),len(estimators)))
    best_rounds = np.zeros ((len(skf),len(estimators)))
    

    
    for j, est in enumerate(estimators):
        print ("Model %d: %s" %(j+1, est))
        test_blend_x_j = np.zeros((test_x.shape[0], len(skf)))
        for i, (train, val) in enumerate(skf):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x[train]
            train_y_fold = train_y[train]
            val_x_fold = train_x[val]
            val_y_fold = train_y[val]
            if early_stopping_rounds==0: # without early stopping
                est.fit(train_x_fold, train_y_fold)
                best_rounds[i,j]=est.n_estimators
                val_y_predict_fold = est.predict(val_x_fold)
                score = log_mae(val_y_fold, val_y_predict_fold,200)
                print ("Score: ", score)
                scores[i,j]=score
                train_blend_x[val, j] = val_y_predict_fold
                test_blend_x_j[:,i] = est.predict(test_x)
                print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))
            else:                        # early stopping
                est.set_params( n_estimators=100000)
                est.fit(train_x_fold,
                        train_y_fold,
                        eval_set=[(val_x_fold, val_y_fold)],
                        eval_metric=lgbm_eval_mae,
                        early_stopping_rounds=early_stopping_rounds,
                        verbose=False
                       )
                best_round=est.best_iteration
                best_rounds[i,j]=best_round
                print ("best round %d" % (best_round))
                val_y_predict_fold = est.predict(val_x_fold,num_iteration=best_round)
                score = log_mae(val_y_fold, val_y_predict_fold,200)
                print ("Score: ", score)
                scores[i,j]=score
                train_blend_x[val, j] = val_y_predict_fold
                test_blend_x_j[:,i] = est.predict(test_x,num_iteration=best_round)
                print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            
   
        test_blend_x[:,j] = test_blend_x_j.mean(1)
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x, scores,best_rounds )

## Load Data

In [ ]:
# Load data
start = time.time() 
train_data = pd.read_csv('../input/train.csv')
train_size=train_data.shape[0]
print ("Loading train data finished in %0.3fs" % (time.time() - start))        

test_data = pd.read_csv('../input/test.csv')
print ("Loading test data finished in %0.3fs" % (time.time() - start))        

In [ ]:
train_data.head(5)

## Merge train and test

This will save our time on duplicating logics for train and test and will also ensure the transformations applied on train and test are the same.

In [ ]:
full_data=pd.concat([train_data
                       ,test_data])
del( train_data, test_data)
print ("Full Data set created.")

## Group features

In this step we will group the features into different groups so we can preprocess them seperately afterward.

In [ ]:
data_types = full_data.dtypes  
cat_cols = list(data_types[data_types=='object'].index)
num_cols = list(data_types[data_types=='int64'].index) + list(data_types[data_types=='float64'].index)

id_col = 'id'
target_col = 'loss'
num_cols.remove('id')
num_cols.remove('loss')

print ("Categorical features:", cat_cols)
print ( "Numerica features:", num_cols)
print ( "ID: %s, target: %s" %( id_col, target_col))

## Categorical features 
### 1. Label Encoding (Factorizing)

In [ ]:
LBL = preprocessing.LabelEncoder()
start=time.time()
for cat_col in cat_cols:
#     print ("Factorize feature %s" % (cat))
    full_data[cat_col] = LBL.fit_transform(full_data[cat_col])
print ('Label enconding finished in %f seconds' % (time.time()-start))


## New: Two-way interactions for categorical features

The idea was borrowed from a kernel script from Kaggle forum and I made following changes to make it run faster and more accurate:

1. Instead of decoding the interactions in a lexical way, I simply concatenated each pair of features and apply label-encoding on them. It ended up with very similar results as the original script but way faster and more memory efficient.

2. Added four features to the list which appeared to be effective. Here are some tips if you want to explore more possibilities:
    * Train a model use original features (no interaction)
    * Count distinct values of each feature
    * Observe the distribution of combination features used in the original script by feature importance and value count.
        * Mean count of values: 3.64,max:17, min:2 which indicates low cardinality features are prefered.
        * Mean feature importance: 
        * It seems low cardinality features (value counts 2 to 4), and midium important features are prefered.
        * There's a high correlation between feature number and value counts (cont77-88). Could it be useful?

In [ ]:

# feature name	importance	count	is in comb
#  cat1	1022	2	TRUE
#  cat10	519	2	TRUE
#  cat100	4301	15	
#  cat101	2474	19	
#  cat102	767	9	
#  cat103	1693	14	TRUE
#  cat104	380	17	
#  cat105	879	20	
#  cat106	489	18	
#  cat107	1055	20	
#  cat108	628	11	
#  cat109	618	85	
#  cat11	270	2	TRUE
#  cat110	2764	134	
#  cat111	1665	17	TRUE
#  cat112	4425	51	
#  cat113	1834	63	
#  cat114	1962	19	
#  cat115	645	23	
#  cat116	1466	349	
#  cat12	310	2	TRUE
#  cat13	217	2	TRUE
#  cat14	98	2	TRUE
#  cat15	2	2	
#  cat16	133	2	TRUE
#  cat17	74	2	
#  cat18	92	2	
#  cat19	114	2	
#  cat20	23	2	
#  cat21	69	2	
#  cat22	28	2	
#  cat23	352	2	TRUE
#  cat24	181	2	TRUE
#  cat25	272	2	TRUE
#  cat26	226	2	
#  cat27	372	2	
#  cat28	208	2	TRUE
#  cat29	196	2	
#  cat3	144	2	TRUE
#  cat30	157	2	
#  cat31	218	2	
#  cat32	56	2	
#  cat33	54	2	
#  cat34	26	2	
#  cat35	14	2	
#  cat36	375	2	TRUE
#  cat37	395	2	
#  cat38	309	2	TRUE
#  cat39	219	2	
#  cat40	175	2	TRUE
#  cat41	168	2	
#  cat42	66	2	
#  cat43	173	2	
#  cat44	292	2	
#  cat45	154	2	
#  cat46	41	2	
#  cat47	27	2	
#  cat48	29	2	
#  cat49	233	2	
#  cat50	328	2	TRUE
#  cat51	77	2	
#  cat52	243	2	
#  cat53	343	2	
#  cat54	196	2	
#  cat55	11	2	
#  cat56	17	2	
#  cat57	168	2	TRUE
#  cat58	24	2	
#  cat59	39	2	
#  cat60	41	2	
#  cat61	65	2	
#  cat62	8	2	
#  cat63	22	2	
#  cat64	1]]	2	
#  cat65	94	2	
#  cat66	215	2	
#  cat67	58	2	
#  cat68	28	2	
#  cat69	16	2	
#  cat7	64	2	TRUE
#  cat70	7	2	
#  cat71	107	2	
#  cat72	848	2	TRUE
#  cat73	746	3	TRUE
#  cat74	192	3	
#  cat75	568	3	
#  cat76	209	3	TRUE
#  cat77	88	4	
#  cat78	116	4	
#  cat79	970	4	TRUE
#  cat80	997	4	TRUE
#  cat81	1004	4	TRUE
#  cat82	1111	4	TRUE
#  cat83	1227	4	
#  cat84	765	4	
#  cat85	136	4	
#  cat86	37	4	
#  cat87	497	4	TRUE
#  cat88	122	4	
#  cat89	8	9	TRUE
#  cat9	323	2	TRUE
#  cat90	34	7	TRUE
#  cat91	1198	8	
#  cat92	384	9	
#  cat93	816	5	
#  cat94	459	7	
#  cat95	392	5	
#  cat96	258	9	
#  cat97	387	7	
#  cat98	163	5	
#  cat99	263	17	



In [ ]:
LBL = preprocessing.LabelEncoder()
start=time.time()

import itertools

# Original combination features
comb_list = ['cat80', 'cat87', 'cat57', 'cat12', 'cat79', 'cat10', 'cat7', 'cat89', 'cat2', 'cat72', 'cat81', 'cat11', 'cat1', 'cat13', 'cat9', 'cat3', 'cat16', 'cat90', 'cat23', 'cat36', 'cat73', 'cat103', 'cat40', 'cat28', 'cat111', 'cat6', 'cat76', 'cat50', 'cat5', 'cat4', 'cat14', 'cat38', 'cat24', 'cat82', 'cat25']
# Added 'cat37','cat27','cat53','cat44'
comb_list = ['cat37','cat27','cat53','cat44'] + comb_list
comb_two_cols = [] 
for comb in itertools.combinations(comb_list, 2):
    comb_col_name = comb[0] + comb[1]
    print (comb_col_name)
    full_data [comb_col_name] = LBL.fit_transform(full_data [ comb[0]] + full_data [ comb[1]])
    comb_two_cols.append(comb_col_name)
    
print ('Label enconding finished in %f seconds' % (time.time()-start))

print (full_data.shape)


### 2. One Hot Encoding (get dummies)

OHE can be done by either Pandas' get_dummies() or SK Learn's OneHotEncoder. 

* get_dummies is easier to implement (can be used directly on raw categorical features, i.e. strings, but it takes longer time and is not memory efficient.

* OneHotEncoder requires the features being converted to numeric, which has already been done by LabelEncoder in previous step, and is much more efficient (4x faster at least).

* We will convert the OHE's results to a sparse matrix which uses way less memory as compared to dense matrix. However, not all algorithms and packagers support sparse matrix, e.g. Keras. In that case, we'll need to use other tricks to make it work.


## Note: due to the large number of newly added two-way features, the memory usage can easily excceed 10G if you want to run OHE, which, however, is optional so I just commented it out. Feel free to uncomment it if you'd like to give it a try.

In [ ]:
# OHE = preprocessing.OneHotEncoder(sparse=True)
# start=time.time()
# full_data_sparse=OHE.fit_transform(full_data[cat_cols+comb_two_cols])
# print ('One-hot-encoding finished in %f seconds' % (time.time()-start))

# print (full_data_sparse.shape)

# ## it should be (313864, 1176)

### 3. Leave-one-out Encoding

This is a very useful trick that has been used by many Kaggle winning solutions. It's particularly effective for high cardinality categorical features, postal code for instance. However, it doesn't seem to help a lot for this competition and the following code is just FYI. Feel free to skip it as it may take long time to run.

In [ ]:
# start=time.time()
# loo_cols =[]
# for col in cat_cols:
#     print ("Leave-One-Out Encoding  %s" % (col))
#     print ("Leave-one-out encoding column %s for %s......" % (col, target_col))
#     aggr=full_data.groupby(col)[target_col].agg([np.mean]).join(full_data[:train_size].groupby(col)[target_col].agg([np.sum,np.size]),how='left')        
#     meanTagetAggr = np.mean(aggr['mean'].values)
#     aggr=full_data.join(aggr,how='left', on=col)[list(aggr.columns)+[target_col]]
#     loo_col = 'MEAN_BY_'+col+'_'+target_col
#     full_data[loo_col] = \
#     aggr.apply(lambda row: row['mean'] if math.isnan(row[target_col]) 
#                                                        else (row['sum']-row[target_col])/(row['size']-1)*random.uniform(0.95, 1.05) , axis=1)
#     loo_cols.append(loo_col)
#     print ("New feature %s created." % (loo_col))
# print ('Leave-one-out enconding finished in %f seconds' % (time.time()-start))

## Numeric features

We will apply two preprocessings on numeric features:

1. Apply box-cox transformations for skewed numeric features.

2. Scale numeric features so they will fall in the range between 0 and 1.

Please be advised that these preprocessings are not necessary for tree-based models, e.g. XGBoost. However, linear or linear-based models, which will be dicussed in following weeks, may benefit from them.

** Calculate skewness of each numeric features: **

In [ ]:
from scipy.stats import skew, boxcox
skewed_cols = full_data[num_cols].apply(lambda x: skew(x.dropna()))
print (skewed_cols.sort_values())

** Apply box-cox transformations: **

In [ ]:
skewed_cols = skewed_cols[skewed_cols > 0.25].index.values
for skewed_col in skewed_cols:
    full_data[skewed_col], lam = boxcox(full_data[skewed_col] + 1)

** Apply Standard Scaling:**

In [ ]:
SSL = preprocessing.StandardScaler()
for num_col in num_cols:
    full_data[num_col] = SSL.fit_transform(full_data[num_col])

#### Note: LBL and OHE are likely exclusive so we will use one of them at a time combined with numeric features. In the following steps we will use OHE + Numeric to tune XGBoost models and you can apply the same process with OHE + Numeric features. Averaging results from two different models will likely generate better results.

** Numberic features + Label Encoded Categorical features

## LE + LightGBM

### New: updated LightGBM blending

Note: 10 folds appear to be much bettern than 4

In [ ]:
## start = time.time()
estimators = [lgb.LGBMRegressor(learning_rate=0.005,                             
                     n_estimators=100000,
                     max_bin=255,
                     num_leaves=70,
                     min_child_samples=230,
                     colsample_bytree=0.3,
                     subsample=0.9,
                     subsample_freq=1,
                     reg_alpha=0.003,
                     silent=False),
              lgb.LGBMRegressor(learning_rate=0.005,                              
                     n_estimators=100000,
                     max_bin=255,
                     num_leaves=70,
                     min_child_samples=230,
                     colsample_bytree=0.25,
                     subsample=0.9,
                     subsample_freq=1,
                     reg_alpha=0.003,
                     silent=False)
              ]
(train_blend_x_gbm_le11,
 test_blend_x_gbm_le11,
 blend_scores_gbm_le11,
 best_rounds_gbm_le11) = lgbm_blend(estimators, train_x, train_y, test_x,
                                 10, ## 10 folds appear to be much better than 4
                                 1000)

print (np.mean(blend_scores_gbm_le1,axis=0))
print (np.mean(best_rounds_gbm_le1,axis=0))
np.savetxt("../input/train_blend_x_gbm_le1.csv",train_blend_x_gbm_le1, delimiter=",")
np.savetxt("../input/test_blend_x_gbm_le1.csv",test_blend_x_gbm_le1, delimiter=",")

## LE + XGBoost

1. As quite a lot new features were added, colsample_bytree needs to be adjusted to a much smaller number.
2. 10 folds tends to be much bettern than 4.

In [ ]:
estimators = [xgb.XGBRegressor(objective=logregobj,
                              learning_rate=0.01,
                              n_estimators=100000,
                              max_depth=,
                              min_child_weight=,
                              colsample_bytree=,
                              subsample=,
                              gamma=,
                              nthread=-1,
                              silent=True,
                              seed=1234
                             )
              ]

(train_blend_x_xgb_le,
 test_blend_x_xgb_le,
 blend_scores_xgb_le,
 best_rounds_xgb_le) = xgb_blend(estimators, 
                                      train_x, 
                                      train_y, 
                                      test_x,
                                      10,
                                      500)

print (np.mean(blend_scores_xgb_le,axis=0))
print (np.mean(best_rounds_xgb_le,axis=0))
np.savetxt("../input/train_blend_x_xgb_le.csv",train_blend_x_xgb_le, delimiter=",")
np.savetxt("../input/test_blend_x_xgb_le.csv",test_blend_x_xgb_le, delimiter=",")

## OHE

In [ ]:
lift = 200

full_data_sparse = sparse.hstack((full_data_sparse
                                  ,full_data[num_cols])
                                 , format='csr'
                                 )
print (full_data_sparse.shape)
train_x = full_data_sparse[:train_size]
test_x = full_data_sparse[train_size:]
train_y = np.log(full_data[:train_size].loss.values + lift)
ID = full_data.id[:train_size].values

xgtrain = xgb.DMatrix(train_x, label=train_y,missing=np.nan) #used for Bayersian Optimization

from sklearn.cross_validation import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, train_size=.80, random_state=1234)

### Following section didn't change except:

1. use 10 folds instead of 4
2. XGB linear appears to be time consuming and doesn't help a lot. So feel free to skip it.

### OHE + LightGBM

### OHE +XGBoost

### OHE + MLP

### Blending

## Recap

1. Added two-way feature interactions
2. Use smaller colsample_by_tree (less than 0.1)
3. Use 10 folds instead of 5 for blending